In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import *
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from numpy import sqrt

In [2]:
df = pd.read_excel("/Users/sahana/Downloads/Important/CSS581-HW2/Residential-Building-Data-Set.xlsx")

In [3]:
df

,START YEAR,START QUARTER,COMPLETION YEAR,COMPLETION QUARTER,V-1,V-2,V-3,V-4,V-5,V-6,...,V-22-lag5,V-23-lag5,V-24-lag5,V-25-lag5,V-26-lag5,V-27-lag5,V-28-lag5,V-29-lag5,V-9(output),V-10(output)
0,81,1,85,1,1,3150.0,920.0,598.5,190,1010.84,...,815.50,1755.00,8002.0,60.74,54.26,2978.26,41407.00,601988.1,2200,410
1,84,1,89,4,1,7600.0,1140.0,3040.0,400,963.81,...,1316.30,8364.78,8393.0,90.95,89.79,11379.37,44835.00,929027.1,5000,1000
2,78,1,81,4,1,4800.0,840.0,480.0,100,689.84,...,765.80,1755.00,4930.0,38.70,32.04,1653.06,37933.00,377828.6,1200,170
3,72,2,73,2,1,685.0,202.0,13.7,20,459.54,...,152.25,1442.31,1456.0,9.73,8.34,686.16,8194.00,122031.7,165,30
4,87,1,90,2,1,3000.0,800.0,1230.0,410,631.91,...,2244.10,9231.76,9286.0,136.60,140.20,9821.00,48260.00,1734973.5,5500,700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,83,4,86,1,20,1350.0,350.0,108.0,80,251.37,...,1176.50,8343.60,8382.0,87.27,85.56,10555.47,33626.25,865878.9,1100,150
368,81,2,82,4,20,600.0,150.0,36.0,60,299.55,...,815.50,1755.00,8013.0,62.78,56.45,3387.72,10646.75,606524.2,740,80
369,84,4,86,3,20,1900.0,430.0,285.0,150,364.41,...,1584.30,8776.71,8799.0,101.00,101.89,13571.80,34474.50,1067772.0,860,220
370,82,3,84,4,20,510.0,160.0,30.6,60,245.28,...,909.20,7930.66,8004.0,73.57,69.26,4756.68,21855.00,669640.3,1100,110


In [4]:
df.info()
df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372 entries, 0 to 371
Columns: 109 entries, START YEAR to V-10(output)
dtypes: float64(94), int64(15)
memory usage: 316.9 KB


Index(['START YEAR', 'START QUARTER', 'COMPLETION YEAR', 'COMPLETION QUARTER',
       'V-1', 'V-2', 'V-3', 'V-4', 'V-5', 'V-6',
       ...
       'V-22-lag5', 'V-23-lag5', 'V-24-lag5', 'V-25-lag5', 'V-26-lag5',
       'V-27-lag5', 'V-28-lag5', 'V-29-lag5', 'V-9(output)', 'V-10(output)'],
      dtype='object', length=109)

In [5]:
data = df
y = data["V-9(output)"]
X = data.drop(["V-10(output)", "V-9(output)"], axis=1)

In [6]:
X

,START YEAR,START QUARTER,COMPLETION YEAR,COMPLETION QUARTER,V-1,V-2,V-3,V-4,V-5,V-6,...,V-20-lag5,V-21-lag5,V-22-lag5,V-23-lag5,V-24-lag5,V-25-lag5,V-26-lag5,V-27-lag5,V-28-lag5,V-29-lag5
0,81,1,85,1,1,3150.0,920.0,598.5,190,1010.84,...,15,733.800000,815.50,1755.00,8002.0,60.74,54.26,2978.26,41407.00,601988.1
1,84,1,89,4,1,7600.0,1140.0,3040.0,400,963.81,...,15,1143.800000,1316.30,8364.78,8393.0,90.95,89.79,11379.37,44835.00,929027.1
2,78,1,81,4,1,4800.0,840.0,480.0,100,689.84,...,15,589.500000,765.80,1755.00,4930.0,38.70,32.04,1653.06,37933.00,377828.6
3,72,2,73,2,1,685.0,202.0,13.7,20,459.54,...,12,197.679557,152.25,1442.31,1456.0,9.73,8.34,686.16,8194.00,122031.7
4,87,1,90,2,1,3000.0,800.0,1230.0,410,631.91,...,14,2220.600000,2244.10,9231.76,9286.0,136.60,140.20,9821.00,48260.00,1734973.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,83,4,86,1,20,1350.0,350.0,108.0,80,251.37,...,15,1082.500000,1176.50,8343.60,8382.0,87.27,85.56,10555.47,33626.25,865878.9
368,81,2,82,4,20,600.0,150.0,36.0,60,299.55,...,15,746.800000,815.50,1755.00,8013.0,62.78,56.45,3387.72,10646.75,606524.2
369,84,4,86,3,20,1900.0,430.0,285.0,150,364.41,...,15,1424.100000,1584.30,8776.71,8799.0,101.00,101.89,13571.80,34474.50,1067772.0
370,82,3,84,4,20,510.0,160.0,30.6,60,245.28,...,14,886.400000,909.20,7930.66,8004.0,73.57,69.26,4756.68,21855.00,669640.3


In [7]:
y

0      2200
1      5000
2      1200
3       165
4      5500
       ... 
367    1100
368     740
369     860
370    1100
371     460
Name: V-9(output), Length: 372, dtype: int64

In [8]:
skf = StratifiedKFold(n_splits=10)

In [9]:
XGBoost_MAE = cross_val_score(xgb.XGBRegressor(),
                                    X,y, scoring='neg_mean_absolute_error', cv=skf, n_jobs=-1)
print("The absolute error (MAE) using XGBOOST is:  " + str(abs(XGBoost_MAE)))
print("The mean the MAE values using XGBOOST obtained:"+ str( abs(XGBoost_MAE).mean()))

XGBoost_RMSE = cross_val_score(xgb.XGBRegressor(),
                                    X,y, scoring='neg_mean_squared_error', cv=skf, n_jobs=-1)

#view RMSE
print("RMSE VALUE USING XGBOOST IS:" + str(sqrt(abs(XGBoost_RMSE).mean())))

/Users/sahana/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


The mean absolute error (MAE) using XGBOOST is:  [ 98.07286192  81.05338609 115.65222828 104.13298818 108.31098773
  93.8296781   80.1939289   68.46759363 119.27493946 125.34335203]
The mean the MAE values using XGBOOST obtained:99.43319443204834


/Users/sahana/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


RMSE VALUE USING XGBOOST IS:210.1706669852071


In [10]:
from sklearn.ensemble import GradientBoostingRegressor
GradientBoostingRegressor_MAE = cross_val_score(GradientBoostingRegressor(),
                                    X,y, scoring='neg_mean_absolute_error', cv=skf, n_jobs=-1)
# force scores to be positive
scores = abs(GradientBoostingRegressor_MAE)
print("The mean absolute error (MAE) using gradient boosting regressor is:  " + "\n" + str(scores))
print("The mean MAE values obtained using gradient boosting regressor: "+ str(scores.mean()))


GradientBoostingRegressor_RMSE = cross_val_score(GradientBoostingRegressor(),
                                    X,y, scoring='neg_mean_squared_error', cv=skf, n_jobs=-1)
GradientBoostingRegressor_RMSE = abs(GradientBoostingRegressor_RMSE)
print("The RMSE VALUE is: "+ str(sqrt(GradientBoostingRegressor_RMSE.mean())))

/Users/sahana/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


The mean absolute error (MAE) using gradient boosting regressor is:  
[ 88.12227973  56.88685731  91.59614563  84.92410545 107.46131409
  77.45123686  68.52320751  68.47122386 103.33787698  94.03671292]
The mean the MAE values obtained using gradient boosting regressor: 84.0810960351781


/Users/sahana/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


The RMSE VALUE is: 176.301119269212


In [11]:
from catboost import CatBoostRegressor
model = CatBoostRegressor(verbose=0, n_estimators=100)
CatBoostRegressor_MAE = cross_val_score(model,
                                    X,y, scoring='neg_mean_absolute_error', cv=skf, n_jobs=-1)
# force scores to be positive
scores = abs(CatBoostRegressor_MAE)
print("The mean absolute error (MAE) using CatBoostRegressor is:")
print(scores)
print("The mean the MAE values obtained:"+ str(scores.mean()))

CatBoostRegressor_RMSE = cross_val_score(model,
                                    X,y, scoring='neg_mean_squared_error', cv=skf, n_jobs=-1)
scores_rmse = abs(CatBoostRegressor_RMSE)

print("The RMSE VALUE is: " + str(sqrt(scores_rmse.mean())))

/Users/sahana/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


The mean absolute error (MAE) using CatBoostRegressor is:
[135.97321292 105.46089991 148.63917287 121.44955788 126.49079334
  88.3075902  108.30982589  92.64120086 120.54156141 146.47148956]
The mean the MAE values obtained:119.42853048494048


/Users/sahana/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


The RMSE VALUE is: 202.95353978779033


Q2 B Selecting 7 features using RFE. RMSE and MAE values reduce thereby improving performance

In [12]:
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor

rfe = RFE(estimator=GradientBoostingRegressor(), n_features_to_select=7)
model = GradientBoostingRegressor()
pipeline = Pipeline(steps=[('s',rfe),('m',model)])
n_scores = cross_val_score(pipeline, X, y, scoring='neg_mean_absolute_error', cv=skf, n_jobs=-1, error_score='raise')
# report performance
print("THE MAE values after feature selection using RFE is: \n" + str(abs(n_scores)))
print("THE MAE values after feature selection using RFE is " + str(abs(n_scores).mean()))

/Users/sahana/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


THE MAE values after feature selection using RFE is: 
[ 87.80899412  49.90602997  88.3718834   71.5251536   90.57321148
  71.77373927  73.20438466  66.94760351 105.92136436 108.76387039]
THE MAE values after feature selection using RFE is 81.4796234764934


In [13]:
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor

rfe = RFE(estimator=GradientBoostingRegressor(), n_features_to_select=7)
model = GradientBoostingRegressor()
pipeline = Pipeline(steps=[('s',rfe),('m',model)])
n_scores_RMSE = cross_val_score(pipeline, X, y, scoring='neg_mean_squared_error', cv=skf, n_jobs=-1, error_score='raise')
n_scores_RMSE = abs(n_scores_RMSE)

# report performance
print("THE RMSE values after feature selection using RFE is " + str(sqrt(n_scores_RMSE).mean()))


/Users/sahana/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


THE RMSE values after feature selection using RFE is 159.27886738813


In [14]:
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline
import xgboost as xgb

rfe = RFE(estimator=xgb.XGBRegressor(), n_features_to_select=7)
model = xgb.XGBRegressor()
pipeline = Pipeline(steps=[('s',rfe),('m',model)])
n_scores = cross_val_score(pipeline, X, y, scoring='neg_mean_absolute_error', cv=skf, n_jobs=-1, error_score='raise')
# report performance
print("THE MAE values after feature selection using RFE is: \n" + str(abs(n_scores)))
print("THE MAE values after feature selection using RFE is " + str(abs(n_scores).mean()))


/Users/sahana/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


THE MAE values after feature selection using RFE is: 
[ 83.02344513  46.80161486  92.60705113  89.33838138  66.67613282
  85.94098024  65.44740007  46.33181515  87.60894363 103.24895457]
THE MAE values after feature selection using RFE is 76.70247189778182


In [15]:
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline
import xgboost as xgb

rfe = RFE(estimator=xgb.XGBRegressor(), n_features_to_select=7)
model = xgb.XGBRegressor()
pipeline = Pipeline(steps=[('s',rfe),('m',model)])
n_scores_RMSE = cross_val_score(pipeline, X, y, scoring='neg_mean_squared_error', cv=skf, n_jobs=-1, error_score='raise')
n_scores_RMSE = abs(n_scores_RMSE)

# report performance
print("THE RMSE values after feature selection using RFE is " + str(sqrt(n_scores_RMSE).mean()))


/Users/sahana/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


THE RMSE values after feature selection using RFE is 162.6827976831512
